In [1]:
%load_ext autoreload
%autoreload 2
is_cuda = True

# General setup and perturbation

In [2]:
%pip install -e .

Obtaining file:///Users/smcaleese/Documents/masters-thesis-code/polyjuice
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for polyjuice_nlp (pyproject.toml) ... done
  Created wheel for polyjuice_nlp: filename=polyjuice_nlp-0.1.5-0.editable-py3-none-any.whl size=5973 sha256=fad040ee48f454aa4529d7f5f862d5a3893534f2b562b837641f81307e8f57c4
  Stored in directory: /private/var/folders/t6/_lt6g5116z9f5127kxxf3qgc0000gn/T/pip-ephem-wheel-cache-a0kjcstj/wheels/25/ab/5a/2c39cb2ced826c744df003583a7e2691ec72e79dc71b9ba517
Successfully built polyjuice_nlp
  Attempting uninstall: polyjuice_nlp
    Found existing installation: polyjuice_nlp 0.1.5
    Uninstalling polyjuice_nlp-0.1.5:
      Successfully uninstalled polyjuice_nlp-0.1.5

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip 

In [3]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 12.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [56]:
from polyjuice import Polyjuice
# initiate a wrapper.
# model path is defaulted to our portable model:
# https://huggingface.co/uw-hai/polyjuice
# No need to change this unless you are using customized model
pj = Polyjuice(model_path="uw-hai/polyjuice", is_cuda=True)

# the base sentence
# text = "I liked the movie."
text = "On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink"

# perturb the sentence with one line:
# When running it for the first time, the wrapper will automatically
# load related models, e.g. the generator and the perplexity filter.

perturbations = pj.perturb(
    orig_sent=text,
    ctrl_code="negation",
    num_perturbations=1,
    perplex_thred=None,
    verbose=True,
)


# return: ['It is bad for kids too.',
# "It 's great for kids.",
# 'It is great even for kids.']

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
INFO:polyjuice.polyjuice_wrapper:Generating on these prompts:
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] [BLANK]
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to s

orig_doc: On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink, ctrl_codes: ['negation'], blanked_sents: {'[BLANK]', "On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is [BLANK]. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [53]:
perturbations

[]

In [22]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

def load_generator(model_path="uw-hai/polyjuice", is_cuda=True):
    return pipeline("text-generation", 
        model=AutoModelForCausalLM.from_pretrained(model_path),
        tokenizer=AutoTokenizer.from_pretrained(model_path),
        framework="pt", device="cpu")

In [48]:
import torch
import re

generator = load_generator()
# prompts = ["I liked the movie. <|perturb|> [negation] [BLANK] liked the movie."]
prompts = ["On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] [BLANK]"]

def batched_generate(generator, 
    examples, 
    temperature=1, 
    num_beams=None,
    num_return_sequences=3, 
    do_sample=True, 
    batch_size=128, **kwargs):
    preds_list = []
    with torch.no_grad():
        for e in (range(0, len(examples), batch_size)):
            preds_list += generator(
                examples[e:e+batch_size],
                temperature=temperature,
                num_beams=num_beams,
                max_length=1000,
                early_stopping=False,
                num_return_sequences=num_return_sequences, 
                **kwargs
            )
    return preds_list

PERETURB_TOK = "<|perturb|>"

def remove_blanks(text):
    try:
        before, answers = text.split(SEP_TOK)
        answers = [x.strip() for x in answers.split(ANSWER_TOK)][:-1]
        answers = [x if x != EMPTY_TOK else '' for x in answers]
        for a in answers:
            if a == '':
                before = re.sub(r' %s' % re.escape(BLANK_TOK), a, before, count=1)
            else:
                before = re.sub(r'%s' % re.escape(BLANK_TOK), a, before, count=1)
        return before, answers
    except:
        return text, []

def split_ctrl_code(text):
    r = re.search(r'\[(?P<code>[a-z]+)\](?P<text>.+)', text)
    if r:
        return r.group("code").strip(), r.group("text").strip()
    return "", text

def generate_on_prompts(generator, prompts, temperature=1, 
    num_beams=None, do_sample=True, batch_size=128, **kwargs):
    preds_list = batched_generate(generator, prompts,
        temperature=temperature, 
        num_beams=num_beams, 
        do_sample=do_sample, batch_size=batch_size, **kwargs)
    # if len(prompts) == 1:
    #     preds_list = [preds_list]
    preds_list_cleaned = []
    for prompt, preds in zip(prompts, preds_list):
        print(f"preds: {preds}")
        prev_list = set()
        for s in preds:
            # print(f"s: {s}")
            total_sequence = s["generated_text"].split(PERETURB_TOK)[-1]
            normalized, _ = remove_blanks(total_sequence)
            input_ctrl_code, normalized = split_ctrl_code(normalized)
            prev_list.add((input_ctrl_code, normalized))
        preds_list_cleaned.append(list(prev_list))
    return preds_list_cleaned

In [49]:
output = generate_on_prompts(generator, prompts, num_beams=5, batch_size=1)
output

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


preds: [{'generated_text': "On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] [BLANK] [SEP] On a dark, gloomy New Year's Eve, a nurse, her life slowly eases, and joyfully, comes to an abrupt ending, when a drunken girl, played by a chest-puffing John Caffari, whom we first meet as he looses a girlfriend, comes to love for the first time with Tina and how he has lost her, and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has

[[('negation',
   "[BLANK] [SEP] On a dark, gloomy New Year's Eve, a nurse, her life slowly eases, and joyfully, comes to an abrupt ending, when a drunken girl, played by a chest-puffing John Caffari, whom we first meet as he looses a girlfriend, comes to love for the first time with Tina and how he has lost her, and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he has lost himself and how he ha

In [51]:
text = "On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink"
# text = "On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once."
# text = "I liked the movie."

max_attempts = 10
for i in range(max_attempts):
    print(f"Attempt: {i + 1}")
    perturbations = pj.perturb(
        orig_sent=text,
        ctrl_code="negation",
        num_perturbations=3,
        perplex_thred=None,
        verbose=True,
    )
    if perturbations:
        break


INFO:polyjuice.polyjuice_wrapper:Generating on these prompts:
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that [BLANK] Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones 

Attempt: 1
orig_doc: On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink, ctrl_codes: ['negation'], blanked_sents: {"On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that [BLANK] Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also d

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:polyjuice.polyjuice_wrapper:Generating on these prompts:
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David [BLANK] Holm is, or why this nurse wishes to

Attempt: 2
orig_doc: On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink, ctrl_codes: ['negation'], blanked_sents: {"On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David [BLANK] Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:polyjuice.polyjuice_wrapper:Generating on these prompts:
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him

...

Attempt: 4
orig_doc: On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink, ctrl_codes: ['negation'], blanked_sents: {"On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also dir

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:polyjuice.polyjuice_wrapper:Generating on these prompts:
INFO:polyjuice.polyjuice_wrapper: | On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink <|perturb|> [negation] On a dark, gloomy New Year's Eve night, an ill [BLANK], her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see h

Attempt: 5
orig_doc: On a dark, gloomy New Year's Eve night, an ill nurse, her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also directed) and two of his drunken associates merrily await the coming of the New Year. Here we can tell just when to drink, ctrl_codes: ['negation'], blanked_sents: {"On a dark, gloomy New Year's Eve night, an ill [BLANK], her life slowly ebbing away, demands that David Holm be presented to her at once. We don't yet know who David Holm is, or why this nurse wishes to see him, but her only dying wish is to speak with him just one more time. On the other side of the town, nestled comfortably amongst the gravestones of the local cemetery, Holm (Victor Sjöström, who also d

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


KeyboardInterrupt: 

In [6]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model='gpt2')

# Define the prompt
prompt = "It is great for kids"

# Generate text
generated_texts = generator(prompt, max_length=50, num_return_sequences=3, temperature=0.7, top_k=50, top_p=0.9)

# Print the generated texts
for i, generated_text in enumerate(generated_texts):
    print(f"Generated Text {i + 1}: {generated_text['generated_text']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Text 1: It is great for kids to have a strong connection with their teacher and family members. It is a great way to connect with teachers and parents. We're trying to get more parents involved in education so that they can participate in the learning process. We
Generated Text 2: It is great for kids to have a good life," he said. "We are the only group that doesn't have to deal with all that stuff."

The group also plans to expand the school district's own school day care program to include
Generated Text 3: It is great for kids, but it's not good for adults," he says.

He says that for parents who have children, the idea that their children are "being cared for" is not very different from the idea that they are being


In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

model_path = "uw-hai/polyjuice"
generator = pipeline("text-generation", 
    model=AutoModelForCausalLM.from_pretrained(model_path), 
    tokenizer=AutoTokenizer.from_pretrained(model_path),
    framework="pt")

prompt_text = "A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman."
generator(prompt_text, num_beams=3, num_return_sequences=3)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not being held by [ANSWER] '},
 {'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not being cradled by [ANSWER] '},
 {'generated_text': 'A dog is embraced by the woman. <|perturb|> [negation] A dog is [BLANK] the woman. [SEP] not being held [ANSWER] '}]

In [8]:
# initiate a wrapper.
# model path is defaulted to our portable model:
# https://huggingface.co/uw-hai/polyjuice
# No need to change this unless you are using customized model
from polyjuice import Polyjuice
pj = Polyjuice(model_path="uw-hai/polyjuice", is_cuda=True)

In [9]:
# the base sentence
text = "It is great for kids."

# perturb the sentence with one line:
# When running it for the first time, the wrapper will automatically
# load related models, e.g. the generator and the perplexity filter.
perturbations = pj.perturb(text)
perturbations

INFO:polyjuice.polyjuice_wrapper:Setup Polyjuice.
INFO:polyjuice.polyjuice_wrapper:Setup SpaCy processor.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_toke

'hello'

In [10]:
# To perturb with more controls,

perturbations = pj.perturb(
    orig_sent=text,
    # can specify where to put the blank. Otherwise, it's automatically selected.
    # Can be a list or a single sentence.
    # blanked_sent=["It is [BLANK] for kids.", "It is great for [BLANK]."],
    # can also specify the ctrl code (a list or a single code.)
    # The code should be from 'resemantic', 'restructure', 'negation', 'insert', 'lexical', 'shuffle', 'quantifier', 'delete'.
    ctrl_code="negation",
    # Customzie perplexity score. 
    perplex_thred=20,
    # number of perturbations to return
    num_perturbations=3,
    # the function also takes in additional arguments for huggingface generators.
    num_beams=3
)
perturbations

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'hello'

In [11]:
# detect the ctrl code from a given sentence pair
pj.detect_ctrl_code("it's great for kids.", 'It is great for kids but not for any adults.')

'negation'

# Select for diversity

Having each perturbation be represented by its token changes, control code, and dependency tree strcuture, we greedily select the ones that are least similar to those already selected. This tries to avoid redundancy in common perturbations such as black -> white.


In [12]:
# over-generate some examples

orig_text = "It is great for kids."
perturb_texts = pj.perturb(
    orig_sent=orig_text, perplex_thred=10, num_perturbations=None, num_beams=3)
orig_and_perturb_pairs = [(orig_text, perturb_text) for perturb_text in perturb_texts]
orig_and_perturb_pairs

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

[('It is great for kids.', 'h'),
 ('It is great for kids.', 'e'),
 ('It is great for kids.', 'l'),
 ('It is great for kids.', 'l'),
 ('It is great for kids.', 'o')]

In [13]:
sampled = pj.select_diverse_perturbations(
    orig_and_perturb_pairs=orig_and_perturb_pairs, nsamples=3)
sampled

/Users/smcaleese/Documents/masters-thesis-code/.env/lib/python3.12/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/smcaleese/Documents/masters-thesis-code/.env/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[('It is great for kids.', 'h'),
 ('It is great for kids.', 'e'),
 ('It is great for kids.', 'l')]

# Select surprising perturbations as counterfactual explanations

Because different models/explainers may have different forms of predictions/feature weight computation methods, Polyjuice selection expects all predictions and feature weights to be precomputed. Here, we give an example of Quora Question Pair Detection. 


In [ ]:
# set a perturbation base
orig = (
    "How can I help a friend experiencing serious depression?",
    "How do I help a friend who is in depression?"
)
orig_label = 1

# we perturb the second question.

In [ ]:
# get a model
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
model_name = "textattack/bert-base-uncased-QQP"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
# sentiment analysis is a general name in Huggingface to load the pipeline for text classification tasks.
# set device=-1 if you don't have a gpu
pipe = pipeline(
    "sentiment-analysis", model=model, tokenizer=tokenizer, 
    framework="pt", device=0 if is_cuda else -1, return_all_scores=True)

In [ ]:
# some wrapper for prediction
import numpy as np
def extract_predict_label(raw_pred):
    raw_pred = sorted(raw_pred, key=lambda r: -r["score"])
    if raw_pred:
        return raw_pred[0]["label"]
    return None
def predict(examples, predictor, batch_size=128):
    raw_preds, preds, distribution = [], [], []
    with torch.no_grad():
        for e in (range(0, len(examples), batch_size)):
            raw_preds.extend(predictor(examples[e:e+batch_size]))
    for raw_pred in raw_preds:
        raw_pred = raw_pred if type(raw_pred) == list else [raw_pred]
        for m in raw_pred:
            m["label"] = int(m["label"].split("_")[1])
    return raw_preds

p = predict([orig], predictor=pipe)[0]
(p, extract_predict_label(p))

In [ ]:
## collect some base perturbations
from polyjuice.generations import ALL_CTRL_CODES

# perturb the second question in orig.
perturb_idx = 1
perturb_texts = pj.perturb(
    orig[perturb_idx], 
    ctrl_code=ALL_CTRL_CODES, 
    num_perturbations=None, perplex_thred=10)

perturb_texts


In [ ]:
# To estimate feature importance, we set up shap explainer
# install shap
!pip install shap

In [ ]:
import shap
import functools
from copy import deepcopy
# setup a prediction function for computing the shap feature importance

def wrap_perturbed_instances(perturb_texts, orig, perturb_idx=1):
    perturbs = []
    for a in perturb_texts:
        curr_example = deepcopy(list(orig))
        curr_example[perturb_idx] = a
        perturbs.append(tuple(curr_example))
    return perturbs

def predict_on_perturbs(perturb_texts, orig, predictor, perturb_idx=1):
    perturbs = wrap_perturbed_instances(perturb_texts, orig, perturb_idx)
    perturbs_preds = predict(perturbs, predictor=predictor)
    perturbs_pred_dicts = [{p["label"]: p["score"] for p in perturbs_pred} for perturbs_pred in perturbs_preds]
    orig_preds = predict([orig], predictor=predictor)
    orig_pred = extract_predict_label(orig_preds[0])
    # the return is probability of the originally predicted label
    return [pr_dict[orig_pred] for pr_dict in perturbs_pred_dicts]
def normalize_shap_importance(features, importances, is_use_abs=True):
    normalized_features = {}
    for idx, (f, v) in enumerate(zip(features, importances)):
        f = f.strip('Ġ')
        if not f.startswith("##"): 
            key, val = "", 0
        key += f.replace("#", "").strip()
        val += v
        if (idx == len(features)-1 or (not features[idx+1].startswith("##"))) and key != "":
            normalized_features[key] = abs(val) if is_use_abs else val
    return normalized_features
def explain_with_shap(orig, predictor=pipe, tokenzier=pipe.tokenizer, perturb_idx=1):
    predict_for_shap_func = functools.partial(
        predict_on_perturbs, orig=orig, predictor=predictor, perturb_idx=perturb_idx)
    shap_explainer = shap.Explainer(predict_for_shap_func, tokenizer) 
    exp = shap_explainer([str(orig[perturb_idx])])
    return normalize_shap_importance(exp.data[0], exp.values[0])

feature_importance_dict = explain_with_shap(orig)
feature_importance_dict

In [ ]:
# get the predictions for original and also new instances
orig_pred = predict([orig], predictor=pipe)[0]

perturb_instances = wrap_perturbed_instances(perturb_texts, orig, perturb_idx)
perturb_preds = predict(perturb_instances, predictor=pipe)

surprises = pj.select_surprise_explanations(
    orig_text=orig[perturb_idx], 
    perturb_texts=perturb_texts, 
    orig_pred=orig_pred, 
    perturb_preds=perturb_preds, 
    feature_importance_dict=feature_importance_dict)
surprises

## Change the context length

In [11]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, GPT2Config

model_path = "uw-hai/polyjuice"
new_context_length = 2048

config = GPT2Config.from_pretrained(model_path)
config.n_positions = new_context_length
# config.n_ctx = new_context_length

model = AutoModelForCausalLM.from_pretrained(model_path, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_path)

pipeline = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer
)


RuntimeError: Error(s) in loading state_dict for GPT2LMHeadModel:
	size mismatch for transformer.wpe.weight: copying a param with shape torch.Size([1024, 768]) from checkpoint, the shape in current model is torch.Size([2048, 768]).
	You may consider adding `ignore_mismatched_sizes=True` in the model `from_pretrained` method.

In [7]:
text = "The quick brown fox"
output = pipeline(text)
output

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'The quick brown fox can be distinguished from smooth brown fox by the dark brown irregular spots. <|perturb|> [resemantic] The quick brown [BLANK] brown [BLANK] by the dark brown irregular spots. [SE'}]

In [9]:
config

GPT2Config {
  "_name_or_path": "/home/wtshuang/datasets/models//gpt2_polyjuice/checkpoint-213650",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.42.3",
  "use_cache": true,
  "vocab_size": 50257
}